In [1]:
!mkdir -p ~/.cloudvolume/secrets
!cp *.json ~/.cloudvolume/secrets/

In [2]:
!pip install cloud-volume
!pip install tifffile

You are using pip version 9.0.3, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F 
import torch.optim as optim

from cloudvolume import CloudVolume
from cloudvolume.lib import Bbox
import numpy as np
import pandas as pd

import tifffile

In [4]:
# helper operations 
def conv3x3(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels,
        kernel_size=3, stride=1, padding=1, bias=True)

def maxpool2x2():
    return nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

class UpConv2x2(nn.Module):
    def __init__(self, channels):
        super(UpConv2x2, self).__init__()
        self.upsample = nn.Upsample(scale_factor=2)
        self.conv = nn.Conv2d(channels, channels // 2,
            kernel_size=2, stride=1, padding=0, bias=True)

    def forward(self, x):
        x = self.upsample(x)
        x = F.pad(x, (0,1,0,1))
        x = self.conv(x)
        return x 
        
def concat(xh, xv):
    return torch.cat([xh, xv], dim=1)


# unet blocks
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(ConvBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class DownConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(DownConvBlock, self).__init__()
        self.maxpool = maxpool2x2()
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, x):
        x = self.maxpool(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class UpConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(UpConvBlock, self).__init__()
        self.upconv = UpConv2x2(in_channels)
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, xh, xv):
        """
        Args:
            xh: torch Variable, activations from same resolution feature maps (gray arrow in diagram)
            xv: torch Variable, activations from lower resolution feature maps (green arrow in diagram)
        """
        xv = self.upconv(xv)
        x = concat(xh, xv)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        fs = [16,32,64,128,256]
        self.conv_in = ConvBlock(1, fs[0])
        self.dconv1 = DownConvBlock(fs[0], fs[1])
        self.dconv2 = DownConvBlock(fs[1], fs[2])
        self.dconv3 = DownConvBlock(fs[2], fs[3])
        self.dconv4 = DownConvBlock(fs[3], fs[4])

        self.uconv1 = UpConvBlock(fs[4], fs[3])
        self.uconv2 = UpConvBlock(fs[3], fs[2])
        self.uconv3 = UpConvBlock(fs[2], fs[1])
        self.uconv4 = UpConvBlock(fs[1], fs[0])
        self.conv_out = conv3x3(fs[0], 1)

        self._initialize_weights()

    def forward(self, x):
        x1 = self.conv_in(x)
        x2 = self.dconv1(x1)
        x3 = self.dconv2(x2)
        x4 = self.dconv3(x3)
        x5 = self.dconv4(x4)
        x6 = self.uconv1(x4, x5)
        x7 = self.uconv2(x3, x6)
        x8 = self.uconv3(x2, x7)
        x9 = self.uconv4(x1, x8)
        x10 = self.conv_out(x9)
        return x10

    def _initialize_weights(self):
        conv_modules = [m for m in self.modules() if isinstance(m, nn.Conv2d)]
        for m in conv_modules:
          n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
          m.weight.data.normal_(0, np.sqrt(2. / n))

# train utils
def np2var(arr, cuda):
    arr = np.expand_dims(np.expand_dims(arr, 0), 0)
    arr = torch.from_numpy(arr)
    arr = Variable(arr)
    if cuda:
        arr = arr.cuda()
    return arr

def var2np(arr):
    arr = arr.data.cpu().numpy()[0,0]
    return arr            

In [5]:
net = torch.load('basil_folddetector_unet180417')

In [8]:
def detect_folds3(z, src_cv, dst_cv, src_chunk_dim, dst_chunk_dim):
    chunk_x_start = (src_chunk_dim[0]-dst_chunk_dim[0]) // 2
    chunk_y_start = (src_chunk_dim[1]-dst_chunk_dim[1]) // 2
    chunk_x_stop = chunk_x_start + dst_chunk_dim[0]
    chunk_y_stop = chunk_y_start + dst_chunk_dim[1]
    chunk_slice = slice(chunk_x_start, chunk_x_stop), slice(chunk_y_start, chunk_y_stop)
    dst_size = dst_cv.shape
    x_chunks = int(np.ceil(dst_size[0]/dst_chunk_dim[0]))
    y_chunks = int(np.ceil(dst_size[1]/dst_chunk_dim[1]))
    dst_shape = x_chunks*dst_chunk_dim[0], y_chunks*dst_chunk_dim[1]
    
    src_o = src_cv.voxel_offset
    src_s = src_cv.shape
    src_slice = (slice(src_o[0] - chunk_x_start, src_o[0] - chunk_x_start + x_chunks*src_chunk_dim[0]), 
                slice(src_o[1] - chunk_y_start, src_o[1] - chunk_y_start + y_chunks*src_chunk_dim[1]))
    src_shape = (src_slice[0].stop - src_slice[0].start, src_slice[1].stop - src_slice[1].start)
    
    print('Downloading {}'.format(str(z)))
    src_image = src_cv[src_slice + (z,)].astype(np.float32).reshape(src_shape)
    dst_image = np.zeros(dst_shape, np.uint8)
    
    USECUDA=True

    print('Processing {}'.format(str(z)))
    for x in range(x_chunks):
        for y in range(y_chunks):
            # print(((x,y)))
            x_start = x*dst_chunk_dim[0]
            y_start = y*dst_chunk_dim[1]
            dst_x_stop = x_start + dst_chunk_dim[0]
            dst_y_stop = y_start + dst_chunk_dim[1]
            dst_slice = slice(x_start, dst_x_stop), slice(y_start, dst_y_stop)
            
            src_x_stop = x_start + src_chunk_dim[0]
            src_y_stop = y_start + src_chunk_dim[1]
            src_slice = slice(x_start, src_x_stop), slice(y_start, src_y_stop)
        
            img = src_image[src_slice]
            # print(img.shape)
            img = img - np.mean(img) # normalize inputs
            img = img / (np.std(img) + 1e-6)
            img = np2var(img, cuda=USECUDA)
            o = net(img)
            dst_image[dst_slice] = var2np(o)[chunk_slice].astype(np.uint8)
        
    dst_image = dst_image.astype(np.uint8)
    print('Uploading {}'.format(str(z)))
    dst_cv[:,:,z] = dst_image[:dst_size[0],:dst_size[1],np.newaxis, np.newaxis]

In [9]:
src_cv = CloudVolume('gs://neuroglancer/basil_v0/raw_image_cropped', mip=5, bounded=False, fill_missing=True, parallel=1)
dst_cv = CloudVolume('gs://neuroglancer/basil_v0/raw_image_cropped/mask/fold_detector_v1', mip=5, cdn_cache=False, bounded=False, autocrop=False, parallel=1)

In [11]:
src_chunk_dim = (512, 512)
dst_chunk_dim = (512-128, 512-128)

In [15]:
for z in range(481,998):
    detect_folds3(z, src_cv, dst_cv, src_chunk_dim, dst_chunk_dim)
for z in range(0,127):
    detect_folds3(z, src_cv, dst_cv, src_chunk_dim, dst_chunk_dim)
for z in range(137,301):
    detect_folds3(z, src_cv, dst_cv, src_chunk_dim, dst_chunk_dim)

Downloading: 140it [00:04, 21.25it/s]                       


Processing 481


Rechunking image:   1%|▏         | 1/72 [00:00<00:11,  6.17it/s]

Uploading 481


Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

...

Downloading: 134it [00:03, 21.70it/s]                        


Processing 300


Rechunking image:   0%|          | 0/72 [00:00<?, ?it/s]

Uploading 300


Uploading:   0%|          | 0/72 [00:00<?, ?it/s]


In [18]:
src_cv = CloudVolume('gs://neuroglancer/basil_v0/raw_image_cropped/permuted', mip=5, bounded=False, fill_missing=True, parallel=1)
dst_cv = CloudVolume('gs://neuroglancer/basil_v0/raw_image_cropped/permuted/mask/fold_detector_v1', mip=5, cdn_cache=False, bounded=False, autocrop=False, parallel=1)

In [19]:
for z in range(127,137):
    detect_folds3(z, src_cv, dst_cv, src_chunk_dim, dst_chunk_dim)

Downloading:  31%|███       | 22/72 [00:00<00:00, 218.65it/s]

Downloading: 136it [00:03, 24.95it/s]                        


Processing 127


Rechunking image:   3%|▎         | 2/72 [00:00<00:05, 13.41it/s]

Uploading 127


...

Downloading: 134it [00:10, 13.56it/s]                        


Processing 136


Rechunking image:   0%|          | 0/72 [00:00<?, ?it/s]

Uploading 136


Uploading:   0%|          | 0/72 [00:00<?, ?it/s]
